In [3]:
import pandas as pd
import numpy as np

from gensim.models import Word2Vec
import math
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector,Input
from keras.utils import np_utils
from keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from keras.layers import Bidirectional
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Add,Concatenate,BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed

from keras.engine.topology import Layer, InputSpec
from keras import initializers
from sklearn.model_selection import train_test_split

#參數
MAX_SEQUENCE_LENGTH = 512
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2



print (keras.__version__)

column_names = ['type','title','text']
df = pd.read_csv('./all_after_mapping.tsv',sep='\t',names=column_names)
labels = df['type'].values
labels = np.array(labels)
print(labels.shape)
labels = np_utils.to_categorical(labels)
print(labels.shape)
tokenlizeword = np.load('./tokenlizeword0225_nopunct.npy',allow_pickle=True)
wmodel=Word2Vec.load('./word2vec.model')
print(wmodel)



2.3.1
(35546,)
(35546, 7)
Word2Vec(vocab=266873, size=300, alpha=0.025)


In [4]:
#x_train=文章數*最大字數*300維
x_train = []
for k in range(tokenlizeword.shape[0]):
# every article have 每篇文章最多幾個字 * 300維 embedding matrix
    embedding_matrix = np.zeros((MAX_SEQUENCE_LENGTH,EMBEDDING_DIM))
    for i in range(len(tokenlizeword[k])):
        if(i>=MAX_SEQUENCE_LENGTH):
            break
        embedding_matrix[i] = wmodel[tokenlizeword[k][i]]
    x_train.append(embedding_matrix)

print("embedding done")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


embedding done


In [5]:
put = np.load('./XML_data.npy')
# put= []
# for k in range(tokenlizeword.shape[0]):
#     put_matrix=np.zeros((EMBEDDING_DIM,MAX_SEQUENCE_LENGTH))
#     for i in range(EMBEDDING_DIM):
#         put_vector=np.zeros(MAX_SEQUENCE_LENGTH)
#         for j in range(MAX_SEQUENCE_LENGTH):
#             put_vector[j]=x_train[k][j][i]
#         put_matrix[i]=put_vector
#     put.append(put_matrix)
#     print(k)
    
# put=np.array(put)
# print(put)
# np.save('XML_data',put)

In [6]:

dev_x = put[:5000]
dev_y = labels[:5000]

test_x = put[5000:10000]
test_y = labels[5000:10000]

train_x = put[10000:]
train_y = labels[10000:]
# !ls
print(labels.shape)
print(dev_x.shape)
print(test_x.shape)
print(train_x.shape)

print("60%")

(35546, 7)
(5000, 300, 512)
(5000, 300, 512)
(25546, 300, 512)
60%


In [7]:

sequence_input = Input(shape=(EMBEDDING_DIM,MAX_SEQUENCE_LENGTH), dtype='float32')
l_cov1= Conv1D(512, 3, activation='relu')(sequence_input)
l_pool1 = MaxPooling1D(3)(l_cov1)
l_cov2 = Conv1D(512, 3, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(3)(l_cov2)
l_cov3 = Conv1D(512, 3, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(3)(l_cov3)  # global max pooling
l_drop2 = Dropout(0.5)(l_pool3)
l_flat = Flatten()(l_drop2)

l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(7, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='Adamax',
              metrics=['acc'])

print("model fitting - simplified convolutional neural network")
model.summary()
model.fit(train_x, train_y, validation_data = (dev_x,dev_y),epochs=5, batch_size=50)
# model.fit(train_x, train_y, validation_split = 0.2,epochs=5, batch_size=128)


model fitting - simplified convolutional neural network
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300, 512)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 298, 512)          786944    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 99, 512)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 97, 512)           786944    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 32, 512)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 30, 512)           786944    
_________________________________________________________________
max

In [8]:
pred=model.predict(test_x,batch_size=50)
pred=np.argmax(pred, axis=1)
print(pred.shape)

(5000,)


In [9]:
#驗證
correct=0 #對的有幾個
for i in range(5000):
    if test_y[i][pred[i]]==1.0:
        correct = correct+1
print(correct)
acc=correct/5000
print(acc)

3831
0.7662
